# JJWXC数据爬取

## 导入模块

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import lxml.html
import pandas as pd
from time import sleep

ModuleNotFoundError: No module named 'bs4'

## 公共部分

In [1]:
startURL = "bookbase_slave.php?orderstr=2"
#proxy = {'http':'175.43.33.31:9999'}
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) \
           Chrome/80.0.3987.122 Safari/537.36',
          'Cookie':'__gads=ID=575608ff3de309c3:T=1576427212:S=ALNI_MaoGfbyodSNLzIqqxbIkqP6jiyTHg; cookieofcommentusername=%25E7%25BB%25B4%25E6%2596%25AF%25E5%259D%25A6%25E6%25A0%25B9%25E7%2589%25B9; UM_distinctid=1724cc23ad43e2-03379a1180d04c-4313f6b-144000-1724cc23ad576b; token=OTE2MDg0NnwyMjhjZTUwMzJjZTlmMTNkNTg1NmM0N2Y5Mzg4ZDM5Y3x8c2hlKioqKioqQDE2My5jb218fDI1OTIwMDB8MXx8fOaZi%2Baxn%2BeUqOaIt3wwfGVtYWls; JJEVER=%7B%22ispayuser%22%3A%229160846-0%22%2C%22foreverreader%22%3A%229160846%22%2C%22sms_total%22%3A%221%22%2C%22fenzhan%22%3A%22noyq%22%7D; __cfduid=df68580c275fc44625713a50586af98601590425726; CNZZDATA30075907=cnzz_eid%3D1784788482-1576425734-null%26ntime%3D1591187240; testcookie=yes; timeOffset_o=491.300048828125; Hm_lvt_bc3b748c21fe5cf393d26c12b2c38d99=1590422200,1591191070; JJSESS=%7B%22nicknameAndsign%22%3A%22undefined%7E%29%2524%22%2C%22clicktype%22%3A%22%22%7D; Hm_lpvt_bc3b748c21fe5cf393d26c12b2c38d99=1591191459'}

In [20]:
df_jjwxc = pd.DataFrame({"author":[], "author_id":[],"novel":[],"novel_id":[], "genre":[],"style":[],"status":[],"counts":[],
                  "scores":[],"post_time":[],"tag":[],"key_word":[],"brief":[],"reviews":[],"collects":[]})

In [21]:
df_jjwxc

,author,author_id,novel,novel_id,genre,style,status,counts,scores,post_time,tag,key_word,brief,reviews,collects


In [23]:
def getNextLink(pageURL):
    global headers
    req = requests.get("http://www.jjwxc.net/"+pageURL, headers=headers)
    req.encoding = 'gbk' #r.apparent_encoding(utf-8) 会出现�乱码问题
    bsObj = bs(req.text,'lxml')
    try:
        nextLink = bsObj.body.find("a", string="下一页").attrs["href"]
        return nextLink
    except AttributeError as e:
        print("已检索至尾页")

In [24]:
#def getLink(pageBsObj):
    #try:
        #nextLink = pageBsObj.body.find("a", string="下一页").attrs["href"]
        #return nextLink
    #except AttributeError as e:
        #print("已检索至尾页")

In [25]:
print(getNextLink("bookbase_slave.php?orderstr=2"))

/bookbase_slave.php?booktype=&opt=&page=2&endstr=&orderstr=2


In [26]:
count = 0

In [27]:
count

0

In [28]:
def getItemData(line):
    global df_jjwxc
    
    #获取作者id
    author_id = line.find_all("a")[0]["href"].split("=")[-1]
    #获取作品详情
    novel_id = line.find_all("a")[1]["href"].split("=")[-1] #作品id
    novelURL = "http://www.jjwxc.net/onebook.php?novelid=" + novel_id
    
    with requests.get(novelURL, stream=True, headers=headers) as req_details: #WinError 10060
        req_details.encoding = 'gbk' 
        novel_page = bs(req_details.text,'lxml')
        #内容标签，list类型：tags
        ts = novel_page.body
        if ts is not None and ts.find_all("a",{"style":"text-decoration:none;color: red;"}) is not None:
            ts = ts.find_all("a",{"style":"text-decoration:none;color: red;"})
            tags=[]
            for t in ts:
                tags.append(t.get_text())
        else:
            tags = []

        #搜索关键词，dict类型：keywords
        keyword = novel_page.body
        if keyword is not None and keyword.find("span",{"class":"bluetext"}) is not None:
            keyword = keyword.find("span",{"class":"bluetext"}).get_text().replace("搜索关键字：","")
            key=keyword.split("┃")
            if (len(key)==3):
                zj = key[0].replace("主角：","")
                pj = key[1].replace("配角：","")
                qt = key[2].replace("其它：","")
                keywords = {'主角':zj,'配角':pj,'其它':qt}
            else:
                keywords = {}
        else:
            keywords = {}

        #一句话简介，str类型：brief
        brief = novel_page.body
        if brief is not None and brief.find("span",{"style":"color:#F98C4D"}) is not None:
            brief = brief.find("span",{"style":"color:#F98C4D"}).get_text().replace("一句话简介：","")
        else:
            brief = ""

        #总书评数
        review_count = novel_page.body
        if review_count is not None and review_count.find("span",{"itemprop":"reviewCount"}) is not None:
            review_count = review_count.find("span",{"itemprop":"reviewCount"}).get_text()
        else:
            review_count = ""

        #当前被收藏数
        collected_count = novel_page.body
        if collected_count is not None and collected_count.find("span",{"itemprop":"collectedCount"}) is not None:
            collected_count = collected_count.find("span",{"itemprop":"collectedCount"}).get_text()
        else:
            collected_count = ""
    
    #作品概述
    #作者名
    author = line.find_all("td")[0].get_text()
    #书名
    novel = line.find_all("a")[1].get_text()
    #类型
    genre = line.find_all("td")[2].get_text().replace(" ","").replace('\n', '').replace('\r', '')
    #风格
    style = line.find_all("td")[3].get_text().strip(" ")
    #进度
    status = line.find_all("td")[4].get_text().replace(" ","").replace('\n', '').replace('\r', '')
    #字数
    word_count = line.find_all("td")[5].get_text()
    #作品积分
    score = line.find_all("td")[6].get_text()
    #发表时间
    post_time = line.find_all("td")[7].get_text().split(" ")[0]
    s = [author, author_id, novel, novel_id, genre, style, status, word_count, score, post_time, tags, keywords, brief, review_count, collected_count]
    s = pd.Series(s, index=['author','author_id','novel','novel_id','genre','style','status','counts','scores','post_time',
                                                            "tag","key_word","brief","reviews","collects"])
    df_jjwxc= df_jjwxc.append(s, ignore_index=True)

In [29]:
def getPageData(pageURL):
    global headers
    global count
    if pageURL is not None:
        req = requests.get("http://www.jjwxc.net/"+pageURL, headers=headers)
        req.encoding = 'gbk' #r.apparent_encoding(utf-8) 会出现�乱码问题
        bsObj = bs(req.text,'lxml')
        nextLink = getNextLink(pageURL)

        lines = bsObj.find_all("tr",{"onmouseover":"this.bgColor = '#ffffff';"})

        for line in lines:
            getItemData(line)
        
        count = count + 1
        print("第{0}页已完成".format(count))
        
        sleep(5)
        getPageData(nextLink)      

In [30]:
getPageData(startURL)

第1页已完成
第2页已完成
第3页已完成
第4页已完成
第5页已完成
第6页已完成
第7页已完成
第8页已完成
第9页已完成
第10页已完成
第11页已完成
第12页已完成
第13页已完成
第14页已完成
第15页已完成
第16页已完成
第17页已完成
第18页已完成
第19页已完成
第20页已完成
第21页已完成
第22页已完成
第23页已完成
第24页已完成
第25页已完成
第26页已完成
第27页已完成
第28页已完成
第29页已完成
第30页已完成
第31页已完成
第32页已完成
第33页已完成
第34页已完成
第35页已完成
第36页已完成
第37页已完成
第38页已完成
第39页已完成
第40页已完成
第41页已完成
第42页已完成
第43页已完成
第44页已完成
第45页已完成
第46页已完成
第47页已完成
第48页已完成
第49页已完成
第50页已完成
第51页已完成
第52页已完成
第53页已完成
第54页已完成
第55页已完成
第56页已完成
第57页已完成
第58页已完成
第59页已完成


ConnectionError: HTTPConnectionPool(host='www.jjwxc.net', port=80): Max retries exceeded with url: /onebook.php?novelid=3522956 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001C03B3F9288>: Failed to establish a new connection: [Errno 11004] getaddrinfo failed'))

In [31]:
df_jjwxc.index = df_jjwxc.index+1

In [32]:
df_jjwxc

,author,author_id,novel,novel_id,genre,style,status,counts,scores,post_time,tag,key_word,brief,reviews,collects
1,墨香铜臭,1322620,天官赐福,3200611,原创-纯爱-架空历史-爱情,轻松,已完成,1144742,33430495232,2017-05-26,"[灵异神怪, 情有独钟, 仙侠修真, 励志人生]","{'主角': '谢怜（受），花城（攻） ', '配角': ' ', '其它': ' 花怜，...",为你，所向披靡！,1714888,1536335
2,木苏里,966799,全球高考,3419133,原创-纯爱-近代现代-剧情,轻松,已完成,589514,21592004608,2017-12-03,"[强强, 无限流, 相爱相杀, 未来架空]","{'主角': '游惑，秦究 ', '配角': ' 于闻，狄黎，舒雪，吴俐 ', '其它': ...",并肩炸考场,569943,813240
3,木苏里,966799,某某,3173202,原创-纯爱-近代现代-爱情,正剧,已完成,437995,18527649792,2017-04-27,"[花季雨季, 情有独钟, 破镜重圆]","{'主角': '盛望，江添 ', '配角': ' ', '其它': ' '}",欢迎到江苏来高考,488514,591937
4,巫哲,570049,撒野,2956313,原创-纯爱-近代现代-爱情,轻松,已完成,899013,17990995968,2016-10-15,[强强],"{'主角': '蒋丞，顾飞 ', '配角': ' 无所谓啦反正你们都记不住的 ', '其它'...",来啊，撒野呀，反正有大把时光,397134,929111
5,漫漫何其多,828396,AWM[绝地求生],3468871,原创-纯爱-近代现代-爱情,轻松,已完成,330466,17680035840,2018-01-24,"[情有独钟, 破镜重圆, 竞技, 甜文]","{'主角': '祁醉，于炀 ', '配角': ' 卜那那，贺小旭，老凯，赖华，辛巴 ', '...",你是我的可遇不可求,507462,837939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2963,雨落轻尘,1067472,和精分皇帝he了,3488851,原创-纯爱-幻想未来-爱情,正剧,已完成,420429,566790400,2018-02-10,"[生子, 虐恋情深, 相爱相杀, 星际]","{'主角': '盛东阳 ', '配角': ' 凯撒.塞特.卡米希尔 ', '其它': ' '}",精分皇帝爱上我！,13804,22679
2964,狂上加狂,103563,危宫惊梦,2315675,原创-言情-古色古香-爱情,正剧,已完成,586264,566745856,2014-12-12,"[, 宫廷侯爵, 宫斗]","{'主角': '卫冷侯，聂清麟 ', '配角': ' ', '其它': ' '}",收服邪魅狂狷佞臣之路,13870,26044
2965,凉蝉,1244319,非正常海域,3435797,原创-纯爱-近代现代-爱情,正剧,已完成,510376,566643008,2017-12-22,"[强强, 现代架空, 悬疑推理, 异闻传说]","{'主角': '这个哨兵，那个向导 ', '配角': ' 普通人类，特殊人类 ', '其它'...",让我给你洗洗脑子,16256,40951
2966,萝卜兔子,468959,重生之银河巨星,1287855,原创-纯爱-幻想未来-爱情,轻松,已完成,952721,566574080,2011-08-24,"[强强, 重生]","{'主角': '西泽（郑西泽） ', '配角': ' 阿诺德 ', '其它': ' 爽爽爽爽'}",影帝奋斗在未来娱乐圈,14587,27516


In [33]:
#df_jjwxc.to_csv("jjwxc2967.csv", encoding="utf_8_sig")